Этапы задания: 

1. Установите git и dvc. 
2. Создайте папку lab4 в корне проекта. 
3. Настройте папку проекта для работы с git и dvc. 
4. Настройте git репозиторий. 
5. Настройте удаленное хранилище файлов, например на Google Disk или S3. 
6. Создайте датасет о пассажирах “Титаника”, например, catboost.titanic(). 
7. Создайте датасет, в котором содержится информация о классе (“Pclass”),  поле (“Sex”) и возрасте (“Age”) пассажира. Сделайте коммит в git и push в dvc. 
8. Создайте новую версию датасета, в котором пропущенные (nan) значения в поле “Age” будут заполнены средним значением. Сделайте коммит в git и push в dvc. 
9. Создайте новый признак с использованием one-hot-encoding для строкового признака “Пол” (“Sex”). Сделайте коммит в git и push в dvc. 
10. Выполните переключение между всеми созданными версиями датасета. 

## Создание датасета

Папку lab4 и установку библиотек сделали ранее в терминале перед созданием блокнота.

In [1]:
from catboost import datasets
import pandas as pd

train, test = datasets.titanic()
data = pd.concat([train,test])[['Pclass', 'Sex', 'Age']]

In [2]:
data.sample(5)

,Pclass,Sex,Age
144,2,male,18.0
663,3,male,36.0
249,2,male,54.0
114,1,female,63.0
736,3,female,48.0


## Сохранение датасета. Первый пуш в dvc

In [3]:
data.to_csv('titanic_data.csv')

Индексируем датасет в dvc

In [4]:
!dvc add ./titanic_data.csv

                                                                core>⠋ Checking graph
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/home/user/Desktop/UrFU_MLOps_course/.dvc/cache/files/md
                                                                                
!
  0%|          |Adding titanic_data.csv to cache      0/1 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |Checking out /home/user/Desktop/UrFU_M0/1 [00:00<?,    ?files/s]
100% Adding...|████████████████████████████████████████|1/1 [00:00, 27.12file/s]

To track the changes with git, run:

	git add .gitignore titanic_data.csv.dvc

To enable auto staging, run:

	dvc config core.autostage true


Пушим в хранилище

In [5]:
!dvc push

Pushing
!
  0% Checking cache in '1OAkggea2i7Edb7-Bphn6vNSTcB5M0pwt/files/md5'| |0/? [00:0
                                                                                
!
  0% Checking cache in '/home/user/Desktop/UrFU_MLOps_course/.dvc/cache/files/md
                                                                                
!
  0%|          |Pushing to gdrive                     0/1 [00:00<?,     ?file/s]
  0%|          |Pushing to gdrive                     0/1 [00:00<?,     ?file/s]

!

  0%|          |/home/user/Desktop/UrFU_MLOps_c0.00/20.1k [00:00<?,        ?B/s]

 40%|███▉      |/home/user/Desktop/UrFU_ML8.00k/20.1k [00:04<00:06,    2.02kB/s]

                                                                                
100%|██████████|Pushing to gdrive                 1/1 [00:05<00:00,  5.22s/file]
Pushing                                                                         
1 file pushed


Проверим что изменилось в git (всё в текущей директории)

In [6]:
!git status

On branch lab4_dvc
Untracked files:
  (use "git add <file>..." to include in what will be committed)
	./

nothing added to commit but untracked files present (use "git add" to track)


Закоммитим изменения в гит. Видны следующие изменения:
- добавился .gitignore с нашим датасетом, чтоб в гит не улетал
- добавился .dvc файл, в котором идёт индексация коммитов в dvc
- добавился блокнот, в котором мы работаем 

In [8]:
!git add .
!git commit -m "add titanic dataset"

[lab4_dvc ade1d19] add titanic dataset
 3 files changed, 642 insertions(+)
 create mode 100644 lab4_dvc/.gitignore
 create mode 100644 lab4_dvc/titanic_data.csv.dvc
 create mode 100644 lab4_dvc/work_with_data.ipynb


## Второй пуш в dvc. Средний возраст

In [9]:
data.loc[data['Age'].isna(), 'Age'] = data['Age'].mean()
data.to_csv('./titanic_data.csv')

Проверим видит ли dvc изменения

In [8]:
!dvc status

titanic_data.csv.dvc:                                                           
	changed outs:
		modified:           titanic_data.csv


In [10]:
!dvc add titanic_data.csv 

⠋ Checking graph                                       core>
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/home/user/Desktop/UrFU_MLOps_course/.dvc/cache/files/md
                                                                                
!
  0%|          |Adding titanic_data.csv to cache      0/1 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |Checking out /home/user/Desktop/UrFU_M0/1 [00:00<?,    ?files/s]
100% Adding...|████████████████████████████████████████|1/1 [00:00, 36.38file/s]

To track the changes with git, run:

	git add titanic_data.csv.dvc

To enable auto staging, run:

	dvc config core.autostage true


In [ ]:
!dvc push

In [ ]:
!git status

In [ ]:
!git commit -am "fill NA in Age with mean"

## Третий пуш в dvc. OHE пола

In [12]:
data = pd.get_dummies(data, columns=['Sex'])
data.to_csv('titanic_data.csv')

,Pclass,Age,Sex_female,Sex_male
0,3,22.000000,False,True
1,1,38.000000,True,False
2,3,26.000000,True,False
3,1,35.000000,True,False
4,3,35.000000,False,True
...,...,...,...,...
413,3,29.881138,False,True
414,1,39.000000,True,False
415,3,38.500000,False,True
416,3,29.881138,False,True


In [ ]:
!dvc status

In [ ]:
!dvc add titanic_data.csv 

In [ ]:
!dvc push

In [ ]:
!git status

In [ ]:
!git commit -am "OHE Sex column"

In [ ]:
!git push origin lab4_dvc